# 300. Neural machine translation with attention

- [Google Tutorial](https://www.tensorflow.org/tutorials/text/nmt_with_attention?hl=ko) 참조
- Spanish to English translation 을 seq2seq attention model 로 구현  
- 아래의 matrix 는 model 이 문장을 번역할 때 input sequence 의 어느 부분에  attention 했는지 보여준다

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot" width="400">


In [0]:
import tensorflow as tf
tf.__version__

In [0]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time
from tensorflow.keras.utils import plot_model

# Step1. Download and prepare the dataset

- data 는 [Tab-delimited Bilingual Sentence Pairs](http://www.manythings.org/anki/) 에서 download  


    - English + TAB + The Other Language + TAB + Attribution 형식  
        - Hi.	안녕.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #8355888 (Eunhee)
        - May I borrow this book? ¿Puedo tomar prestado este libro?

- 편의상 google cloud 에서 data download 하고 다음의 전처리를 해준다.


1.  Teacher Forcing 용 data 생성
    - target_texts_inputs  : 1 만큼 offset 된 target language sentence $\rightarrow$ `<sos>...`
    - target_texts  : target language sentence  $\rightarrow$ `.....<eos>`


2. 특수문자 제거  


3. 각 sentence 를 maximum 길이로 sequence padding  

In [0]:
# Google cloud 에서 dataset download
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', 
    origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
print(path_to_file)

* unicodedata category - Mn*

<a href="https://graphemica.com/categories/nonspacing-mark"> Unicode Characters»Categories»Nonspacing Mark</a>

In [0]:
# Converts the unicode file to ascii
# NFD - Normalization Form Decomposition (EX. 자모를 분리하여 저장한 후 보여줄 때 합쳐서 보여주는 방식)
# Mn - Nonspacing_Mark
# 문장을 NFD 방식으로 normalize 하고 accent(MN) 을 제거
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # word 와 구둣점 사이에 space 추가 
  # eg: "he is a boy." => "he is a boy ."
  # \1 - first group
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.rstrip().strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

### preprocessing example

In [0]:
print(re.sub(r"([?.!,¿])", r" \1 ", u"May I borrow this book?"))
print(re.sub(r"([?.!,¿])", r" \1 ", u"¿Puedo tomar prestado este libro?"))

In [0]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

### accent 삭제 및 white space 제거
### [ENGLISH, SPANISH] 쌍으로 data 구성

In [0]:
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

In [0]:
lines = open(path_to_file, encoding='UTF-8').read().strip().split('\n')
print(lines[100:105])

word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[100:105]]
print(word_pairs)

In [0]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

### Tokenize
- Tokenizer 의 filters='' 로 해주지 않으면 `<start>`, `<end>` token 이 filtering 되므로 주의

In [0]:
def tokenize(sentences):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(sentences)
    tensor = tokenizer.texts_to_sequences(sentences)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### 수행 시간 감안하여 datase size 축소 (optional)

100,000 개 넘는 sentence 를 모두 사용하면 오래 걸리므로 30,000 개의 sentence 만 가지고 train. (번역품질 악화 trade-off)

In [0]:
num_examples = 30000

input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer \
            = load_dataset(path_to_file, num_examples)

# target tensor 의 max_length 계산
max_length_targ, max_length_inp \
            = max(len(t) for t in target_tensor), max(len(t) for t in input_tensor)
max_length_targ, max_length_inp

In [0]:
# 80-20 split 으로 train, validation set 분리 
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val \
                 = train_test_split(input_tensor, target_tensor, test_size=0.2)

print(len(input_tensor_train), len(target_tensor_train))
print(len(input_tensor_val), len(target_tensor_val))

In [0]:
print ("Input Language; ")     
print([inp_lang_tokenizer.index_word[i] for i in input_tensor_train[0] if i != 0])
print ()
print ("Target Language; ")
print([targ_lang_tokenizer.index_word[i] for i in target_tensor_train[0] if i != 0])

### tf.data dataset 생성

In [0]:
BUFFER_SIZE = len(input_tensor_train)   # 24,000
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE    #375
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang_tokenizer.word_index) + 1       #9413+1
vocab_tar_size = len(targ_lang_tokenizer.word_index) + 1      #4934+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [0]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

## Write the encoder and decoder model



<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

input 이 encoder model 을 통과하여 *(batch_size, max_length, hidden_size)* 의 encoder output 과 *(batch_size, hidden_size)* 의 encoder hidden state 가 나온다.

다음의 공식 구현:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="600">

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="600">

여기서는 decoder 에 [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) 사용. 다음과 같이 notation 을 정함.

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

### Decoder 의 pseudo-code 는 다음과 같음:

- BahdanauAttention
    * `score = FC(tanh(FC(EO) + FC(H)))`
    * `attention weights = softmax(score, axis = 1)`   
    score 의 shape 이 *(batch_size, max_length, hidden_size)* 이므로, input 의 length 가 max_length 이다.  
    우리가 하려는 것은 각 input 의 time step에 weight 를 할당하려는 것 이므로, axis=1 에 softmax 를 적용해야 한다.

    * `context vector = sum(attention weights * EO, axis = 1)`. 위와 같은 이유로 axis 1 을 선택.  

- Decoder
    * `embedding output` = decoder input (target language) x 는 embedding layer 를 통과하여 embedding output 으로 출력
    * `merged vector = concat(embedding output, context vector)`
    * merged vector 를 GRU 의 input 으로 전달
    * GRU 의 output 은 dense layer 를 통해 logit 구성
    * GRU 의 hidden 은 next step attention 의 query 로 사용

# Step 2. Encoder model 작성

- keras subclass API 사용  

- attention 작성에는 hidden state 만 필요하므로 LSTM 대신 GRU 사용

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru = tf.keras.layers.GRU(self.enc_units,
                                        return_sequences=True,
                                        return_state=True,
                                        recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, hidden_state = self.gru(x, initial_state = hidden)
        return output, hidden_state

    def initialize_hidden_state(self):   # decoder state 0
        return tf.zeros((self.batch_sz, self.enc_units))  

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('모든 timestep 의 Encoder output shape: \n(batch size, max input sequence length, units) {}'
                       .format(sample_output.shape))
print ('마지막 step 의 hidden state shape: \n(batch size, units) {}'
                       .format(sample_hidden.shape))

In [0]:
encoder.summary()
plot_model(encoder)

### class BahdanauAttention parameters

#### def call(self, query, values):  
- query : (초기값) encoder 의 last hidden state,   
(훈련중) decoder 의 step 별 hidden state (batch_size, hidden_unit) -> (64, 1024)  
- values : encoder 모든 timestep 의 output (batch_size, input_lang_max_length, hidden_unit) --> (64, 16, 1024)  
- hidden_with_time_axis 의 shape == (batch_size, 1, hidden size) -> (64, 1, 1024)  


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  # query - (초기값) encoder 의 last hidden state, (훈련중) decoder 의 step 별 hidden state
  # value - encoder 의 모든 timestep 의 output (64, 16, 1024)
  def call(self, query, values):
    
    # score shape == (batch_size, max_length, 1) -> (64, 16, 1)  
    # values 와 query 값을 mix 하기 위해 last axis 에 1 을 추가 
    hidden_with_time_axis = tf.expand_dims(query, 1)
    
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, input_lang_max_length, 1) -> (64, 16, 1)
    # timestep 에 대해 softmax 적용 (default 는 last exis)
    attention_weights = tf.nn.softmax(score, axis=1)   

    # context_vector(어텐션 값) 의 shape after sum == (batch_size, hidden_size)
    # query step 에 맞는 새로운 context vector 
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)   

    return context_vector, attention_weights

In [0]:
attention_layer = BahdanauAttention(10)
context_vector, attention_weights = attention_layer(sample_hidden, sample_output)

print("Context vector shape: (batch size, hidden_units) {}"
                  .format(context_vector.shape))
print("Attention weights shape: (batch_size, input_lang_max_sequence_length, 1) {}"
                  .format(attention_weights.shape))

### class Decoder parameters

#### def call(self, x, hidden, enc_output):

- x : timestep t 의 이전 timestep output (teacher forcing 은 이전 timestep true value)  
(batch_size, 1) -> (64, 1)  
- hidden : encoder 의 hidden unit (64, 1024)  
- enc_output : encoder 의  모든 timestep 의 output  
(batch_size, input_lang_max_length, hidden_size) -> (64, 16, 1024)

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    
    self.fc = tf.keras.layers.Dense(vocab_size)  # vocab_size - target_lang vocab_size (4935)

    # Attention unit 생성
    self.attention = BahdanauAttention(self.dec_units)        

  def call(self, x, hidden, enc_output):
    
    # self.attention(query, value)
    # context_vector (batch_size, hidden_size) -> (64, 1024)
    # attention_weights (64, 14, 1)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x.shape - (64, 1)
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)   # (64, 1, 256)

    # attention 값 (context vector) 과 timestep t 의 output 을 연결
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) # (64, 1, 1280)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)  # (64, 1, 1024), (64, 1024)

    # (batch_size, hidden_size) 
    output = tf.reshape(output, (-1, output.shape[2])) # (64, 1024)

    x = self.fc(output)  # (batch_size, vocab) -> (64, 4929)

    return x, state, attention_weights

- vocab_tar_size : 4929 
- embedding_dim : 256
- BATCH_SIZE : 64  
- sample_output : encoder 의 모든 timestep 의 output (64, 16, 1024)

In [0]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, target_lang_vocab size) {}'
                       .format(sample_decoder_output.shape))

In [0]:
decoder.summary()
plot_model(decoder)

## Define the optimizer and the loss function

- tf.keras.losses.SparseCategoricalCrossentropy 의 reduction 이 NONE 이면, return shape 은 [batch_size, d0, .. dN-1] 이 된다. 
- 그렇지 않으면 scalar return. 

#### def loss_function(real, pred):     
- real : timestep t 의 target 단어(targ[:, t])
    * shape (batch_size, 1) -> (64, 1)
- pred : decoder 의 예측값 (predictions) 
    * shape (batch_size, target_lang_vocab_size) -> (64, 4929)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):     

  # batch 64 개 record 중 timestep t 에 0 padding 이 아닌 
  # 실제 단어가 존재하는 record 만 True 로 만듦
  mask = tf.math.logical_not(tf.math.equal(real, 0))    

  # [word_index] 에 대한 확률분포 array - (64, ), dtype=float32 
  loss_ = loss_object(real, pred)    

  # mask dtype 을 float32 로 type cast
  mask = tf.cast(mask, dtype=loss_.dtype)   

  loss_ *= mask             # 실제 단어가 존재하는 위치 외에는 모두 0 으로 만든다

  return tf.reduce_mean(loss_) 

In [0]:
# 참고 : tf.math.equal, logical_not
print(tf.math.equal([12, 64, 0, 0, 85, 0, 0], 0))
print(tf.math.logical_not(tf.math.equal([12, 64, 0, 0, 85, 0, 0], 0)))

# tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 의 return 값
loss_ = loss_object(tf.convert_to_tensor([0, 1, 2]), 
                    tf.convert_to_tensor([[.9, .05, .05], 
                                          [.5, .89, .6], 
                                          [.05, .01, .94]]))
print(loss_.numpy())

a = loss_object(tf.convert_to_tensor([0]), tf.convert_to_tensor([.9, .05, .05])).numpy()
b = loss_object(tf.convert_to_tensor([1]), tf.convert_to_tensor([.5, .89, .6])).numpy()
c = loss_object(tf.convert_to_tensor([2]), tf.convert_to_tensor([.05, .01, .94])).numpy()

print(a, b, c)
print((a+b+c)/3)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. *input* 을 *encoder* 에 통과시켜 *encoder output* 과 *encoder hidden state* 를 출력
2. *encoder output*, *encoder hidden state*, *decoder input( start token*) 을 decoder 로 공급
3. decoder 는 *prediction* 과 *decoder hidden state* 를 반환
4. *decoder hidden state* 를 다시 model 로 되돌려지고,  *prediction* 은 loss 계산에 사용
5. decoder 에 공급할 next input 을 *teacher forcing* 으로 결정
6. *Teacher forcing* 은 *target word* 를 *next input*  으로  decoder 에 공급하는 기술이다.
7. final step 은 gradient 를 계산하고 optimizer 에 적용하여  backpropagate 하는 것이다.

### def train_step

- inp (64, 16) - [  1  65  53 ...   0   0   0]
- targ (64, 11) - [   1   82   13  231  449    7    2    0    0    0    0]

- enc_output 의 shape (batch_size, target_lang_max_len, hidden_units) -> (64, 16, 1024)
- enc_hidden 의 shape (batch_size, hidden_units) -> (64, 1024) 

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)
    
    # decoder hidden state 의 초기값은  encoder  last hidden state
    dec_hidden = enc_hidden   

    # dec_input shape (batch_size, 1) -> (64, 1) 의 <start> token 으로 시작
    dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):    # targ.shape = (64, 11)
      # passing enc_output to the decoder
      # decoder_hidden 출력을 next decoder 의 input 으로 update
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)  # (64,) (64, 4929)

      # teacher forcing - predictions 가 아닌 true value 를 
      # next step 의 dec_input 으로 제공
      dec_input = tf.expand_dims(targ[:, t], 1)

  # targ.shape -> (64, 11)
  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 40

start = time.time()
for epoch in range(EPOCHS):

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):

    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} 평균 Loss {:.4f}'.format(epoch+1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format((time.time() - start)/(epoch+1)))

print('Total Time {} sec'.format(time.time() - start))

## Translate

* evaluate 함수는 *teacher forcing* 을 하지 않는 것만 다르고  training loop 와 유사. 
* 각 time step 에서의 decoder input 은  이전 step 의  prediction, hidden state, encoder output 이다.
* model 이 *end token* 을 predict 하면 prediction 종료.
* *attention weights* 를 각 time step 별로 저장.

Note: encoder output 은 input sentence 에 대해 단 한번 계산된다.

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden= [tf.zeros((1, units))]   # units : 1024
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):      # 11
    predictions, dec_hidden, attention_weights  \
            = decoder(dec_input, dec_hidden, enc_out)

    # 시각화를 위해 attention weights 저장 
    attention_weights = tf.reshape(attention_weights, (-1,))
    attention_plot[t] = attention_weights.numpy()

    # greedy 하게 next word 선택
    predicted_id = tf.argmax(predictions[0]).numpy()

    predicted_word = targ_lang_tokenizer.index_word[predicted_id]
    result +=  predicted_word + ' '

    if predicted_word == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model - no teacher-forcing
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
import seaborn as sns

# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(8,6))
  ax = fig.add_subplot(1, 1, 1)
  sns.heatmap(attention, annot=True, fmt=".2f")

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u'hace mucho frio aqui.')

In [0]:
translate(u'esta es mi vida.')

In [0]:
translate(u'¿todavia estan en casa?')